# Notebook 2.0 - Xử lý hàng loạt âm thanh

## Cell 1: Định nghĩa class AudioPreprocessor
Class này chứa các phương thức nâng cao để xử lý âm thanh:
- `__init__`: Khởi tạo với VAD (Voice Activity Detection) và tần số lấy mẫu 16kHz
- `load_audio`: Tải và chuyển đổi âm thanh về mono 16kHz
- `denoise`: Giảm nhiễu trong âm thanh
- `apply_vad`: Phát hiện và chỉ giữ lại các đoạn có tiếng nói
- `extract_one_second`: Trích xuất 1 giây có năng lượng cao nhất
- `normalize_audio`: Chuẩn hóa âm thanh về biên độ [-1, 1]
- `process_audio_file`: Pipeline xử lý chính, kết hợp tất cả các bước trên

In [3]:
import os
import numpy as np
import webrtcvad
import noisereduce as nr
from scipy.io import wavfile
from pydub import AudioSegment
import librosa

class AudioPreprocessor:
    def __init__(self):
        self.vad = webrtcvad.Vad(2)
        self.target_sr = 16000
        self.frame_duration_ms = 30
        
    def load_audio(self, input_path):
        """Load and convert audio to mono 16kHz"""
        audio = AudioSegment.from_wav(input_path).set_channels(1).set_frame_rate(self.target_sr)
        raw_audio = np.array(audio.get_array_of_samples())
        rate = audio.frame_rate
        return raw_audio, rate
        
    def denoise(self, audio, sr):
        """Apply noise reduction"""
        denoised = nr.reduce_noise(y=audio.astype(np.float32), sr=sr)
        if sr != self.target_sr:
            denoised = librosa.resample(denoised, orig_sr=sr, target_sr=self.target_sr)
        return denoised, self.target_sr
        
    def apply_vad(self, audio, sr):
        """Apply Voice Activity Detection"""
        frame_length = int(sr * self.frame_duration_ms / 1000)
        frames = [audio[i:i+frame_length] for i in range(0, len(audio) - frame_length, frame_length)]

        def is_speech(frame):
            int16_frame = (frame * 32768).astype(np.int16)
            return self.vad.is_speech(int16_frame.tobytes(), sr)

        flags = [is_speech(frame) for frame in frames]
        speech_mask = np.repeat(flags, frame_length)
        speech_mask = np.pad(speech_mask, (0, len(audio) - len(speech_mask)), mode='constant')
        return audio * speech_mask
        
    def extract_high_energy_segment(self, audio, sr, window_sec=1.5, stride_ratio=0.2):
        """Extract segment with highest energy"""
        window_len = int(window_sec * sr)
        stride = int(stride_ratio * sr)

        max_energy = 0
        best_segment = None
        for i in range(0, len(audio) - window_len, stride):
            window = audio[i:i+window_len]
            energy = np.sum(window.astype(np.float32)**2)
            if energy > max_energy:
                max_energy = energy
                best_segment = window
                
        return best_segment
        
    def extract_one_second(self, audio, sr):
        """Extract 1 second with highest energy"""
        segment_len = int(1.0 * sr)
        stride = int(0.02 * sr)

        max_energy = 0
        best_start = 0
        for i in range(0, len(audio) - segment_len + 1, stride):
            window = audio[i:i + segment_len]
            energy = np.sum(window.astype(np.float32) ** 2)
            if energy > max_energy:
                max_energy = energy
                best_start = i

        return audio[best_start:best_start + segment_len]
        
    def normalize_audio(self, audio):
        """Apply peak normalization"""
        max_val = np.max(np.abs(audio))
        if max_val > 0:
            audio = audio / max_val * 0.99
        return (audio * 32767).astype(np.int16)
        
    def process_audio_file(self, input_path, output_path):
        """Main processing pipeline"""
        # Load audio
        raw_audio, rate = self.load_audio(input_path)
        
        # Denoise
        denoised_audio, rate = self.denoise(raw_audio, rate)
        
        # VAD
        speech_audio = self.apply_vad(denoised_audio, rate)
        
        # Extract high energy segment
        best_segment = self.extract_high_energy_segment(speech_audio, rate)
        
        # Get 1 second segment
        one_sec_segment = self.extract_one_second(best_segment, rate)
        
        # Normalize
        final_output = self.normalize_audio(one_sec_segment)
        
        # Save
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        wavfile.write(output_path, rate, final_output)
        
        return final_output, rate

def process_audio_file(input_path, output_path):
    """Wrapper function for backward compatibility"""
    processor = AudioPreprocessor()
    return processor.process_audio_file(input_path, output_path)


## Cell 2: Xử lý hàng loạt file âm thanh
Cell này thực hiện:
- Tạo thư mục processed để lưu kết quả
- Duyệt qua tất cả các thư mục lệnh trong thư mục raw
- Với mỗi file WAV:
  - Xử lý qua pipeline đã định nghĩa
  - Lưu kết quả vào thư mục processed tương ứng
  - Xử lý lỗi nếu có
- Hiển thị tiến trình xử lý bằng thanh progress bar

Mục đích của notebook này là:
1. Tự động hóa quá trình xử lý âm thanh cho toàn bộ dataset
2. Áp dụng các kỹ thuật xử lý nâng cao (VAD, năng lượng)
3. Đảm bảo chất lượng và tính nhất quán của dữ liệu
4. Chuẩn bị dữ liệu sạch cho quá trình huấn luyện mô hình

In [2]:
from tqdm import tqdm

os.makedirs("../data/processed", exist_ok=True)

raw_dir = "../data/raw"
for command_dir in tqdm(os.listdir(raw_dir), desc="Processing commands"):
    command_path = os.path.join(raw_dir, command_dir)
    if not os.path.isdir(command_path):
        continue

    processed_command_dir = os.path.join("../data/processed", command_dir)
    os.makedirs(processed_command_dir, exist_ok=True)

    for wav_file in tqdm(os.listdir(command_path), desc=f"Processing {command_dir}", leave=False):
        if not wav_file.endswith('.wav'):
            continue

        input_path = os.path.join(command_path, wav_file)
        output_path = os.path.join(processed_command_dir, wav_file)
        
        try:
            process_audio_file(input_path, output_path)
        except Exception as e:
            print(f"Error processing {input_path}: {str(e)}")

Processing commands:  83%|████████▎ | 10/12 [00:35<00:07,  3.59s/it]